In [42]:
import psycopg2
import os
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd
import logging

load_dotenv()

True

In [43]:
def setup_logger(name, log_file, level=logging.DEBUG):    
    # Create a custom logger
    logger = logging.getLogger(name)
    logger.setLevel(level)

    # Create handlers
    file_handler = logging.FileHandler(log_file)
    file_handler.setLevel(level)

    # Create a formatter and set it for the handler
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)

    # Add the handler to the logger
    logger.addHandler(file_handler)

    return logger

In [44]:
def get_connection(logger):
    try:
        conn = psycopg2.connect(
            dbname=os.getenv("DB_NAME"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            host=os.getenv("DB_HOST"),
            port=os.getenv("DB_PORT")
        )
        print("Connection established successfully!")
        logger.info("Connection established successfully!")
        return conn
    except Exception as e:
        print("Connection failed:", e)
        logger.error("Connection failed:", e)

In [45]:
def getAllPotholesToFix(logger):
    conn = get_connection(logger)

    cursor = conn.cursor()
    # Get today's date in YYYY-MM-DD format
    today = datetime.today().date()
    query = """
        SELECT * FROM potholes 
        WHERE is_fixed = false 
        AND estimated_fix_date = %s
        """
    #Execute query with WHERE clause filtering for is_fixed=False and estimated_fix_date=today
    cursor.execute(query, (today,))

    # Fetch results (if applicable)
    rows = cursor.fetchall()

    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    cursor.close()
    conn.close()

    logger.info("All potholes that need to set to fixed retrieved successfully.")

    return df

In [46]:
logger = setup_logger("FixPotholes", "FixPotholes.log")

df = getAllPotholesToFix(logger)
df

Connection established successfully!


,pothole_id,pothole_size,first_reported_date,number_of_reports,coordinates,is_fixed,updated_at,description,address,is_reported,estimated_fix_date
0,77,Small,2024-07-25 21:50:31.429000+00:00,2,"43.478251097734706, -80.51555633000113",False,2024-07-30 01:20:41.784000+00:00,Left lane,"257 Willowdale Place, Waterloo, Ontario N2J 3M...",False,2024-07-30 00:00:00+00:00
1,78,Medium,2024-07-26 00:18:07.515000+00:00,1,"43.4844125703497, -80.51252269064273",False,2024-07-26 00:18:07.515000+00:00,Left lane,"291 Ferndale Place, Waterloo, Ontario N2J 3X8,...",False,2024-07-30 00:00:00+00:00


In [47]:
def setToFixed(df, logger):
    conn = get_connection(logger)

    cursor = conn.cursor()

    update_query = "UPDATE potholes SET is_fixed = true WHERE pothole_id = %s"

    #both wokr the same to add date
    data = [(row['pothole_id'],) for _, row in df.iterrows()]
    #data = [(datetime.strptime(item['EstimatedFixDate'], '%Y-%m-%d').date(), item['pothole_id'],) for item in estimated_fix_dates]

    #Set to Null again
    #data = [(None, item['pothole_id'],) for item in estimated_fix_dates]

    try:
        cursor.executemany(update_query, data)
        conn.commit()  
        print("Records updated successfully!")
        logger.info("Fix Status updated successfully!")
    except Exception as e:
        print("Failed to update records:", e)
        logger.error("Failed to update records with Fix Status:", e)
        conn.rollback() 

    cursor.close()
    conn.close()

In [48]:
setToFixed(df, logger)

Connection established successfully!
Records updated successfully!
